# homework 3
something 

In [97]:
# imports for the script
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
import numpy as np

### 1. Model Selection for Decision Tree Regression

In [98]:
# load dataset from sklearn
diabetes = load_diabetes()
X = diabetes.data

y = diabetes.target

# print(f"X shape: {X.shape}")
# print(f"y shape: {y.shape}")
# print(diabetes.feature_names)
# print(diabetes.target)

In [99]:
# split data into training, validation and test sets

seed = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=seed, shuffle=True
)

In [100]:
max_depth = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
min_samples_split = [2, 3, 4, 5, 6, 7, 8, 9, 10]
grid = []

for i in max_depth:
    for j in min_samples_split:
        grid.append((i, j))


In [101]:
# finding the best hyperparameters
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
heighest_score = np.inf
best_params = None

for max_d, min_s in grid:
    val_scores = []
    for train_index, val_index in kfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        model = DecisionTreeRegressor(max_depth=max_d, min_samples_split=min_s, random_state=seed)
        model.fit(X_train_fold, y_train_fold)
        val_score = mean_squared_error(y_val_fold, model.predict(X_val_fold))
        val_scores.append(val_score)
    mean = np.mean(val_scores)
    # print(f"Mean: {mean}")
    if mean < heighest_score:
        heighest_score = mean
        best_params = (max_d, min_s)

print(heighest_score)
print(f"Best hyperparameters: {best_params}")

4028.034388364361
Best hyperparameters: (2, 2)


In [102]:
# finding the best hyperparameters using GridSearchCV
gscv = GridSearchCV(
    DecisionTreeRegressor(), 
    param_grid={'max_depth': max_depth, 'min_samples_split': min_samples_split}, 
    cv=kfold, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
    )

gscv.fit(X_train, y_train)
print(gscv.best_params_)

{'max_depth': 2, 'min_samples_split': 2}


In [103]:
model = DecisionTreeRegressor(max_depth=best_params[0], min_samples_split=best_params[1])
model.fit(X_train, y_train)
test_score = mean_squared_error(y_test, model.predict(X_test))
print(f"Test score: {test_score}")

Test score: 3735.4996177389003


### 2. Unbalanced Datasets and Model Selection